In [1]:
import pandas as pd
from json import load
from config.config import lee_archivo_csv, configuracion, diccionario_precios_m2
import src.limpieza_columnas
import src.calcula_columnas

In [2]:
def columnas_limpias(df):
    try:
        lista_variables = configuracion['variables']
        for var in lista_variables:
            limpiador = getattr(src.limpieza_columnas, var)
            try:
                df[var] = df[var].apply(limpiador)    
            except:
                df[var] = None   
        return df[lista_variables]
    except Exception as ex :
        plantilla = "Ocurrió una excepción de tipo {0}. Argumentos:\n{1!r}"
        mensaje = plantilla.format(type(ex).__name__, ex.args)
        print(mensaje)
        print("Error en 'columnas_limpias'")

In [3]:
def variables_a_calcular(columnas):
    lista_variables = configuracion['variables']
    return [var for var in lista_variables if var not in columnas] 

In [4]:
def calcula_variables(lista_var,df):
    for var in lista_var:
        calculador = getattr(src.calcula_columnas, var)
        df = calculador(df)
    return df

In [8]:
df = lee_archivo_csv('./data/datos_pisos.csv')
columnas = list(df.columns)
df_limpio = columnas_limpias(df)
var_a_calcular = variables_a_calcular(columnas)
df_calculado = calcula_variables(var_a_calcular,df_limpio)
df_calculado

/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calcula_columnas.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['precio_m2'] = df.cusec.apply(saca_precio)
/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calcula_columnas.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NS'] = df.orientacion.apply(es_NS)
/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calcula_columnas.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,precio,gastos_comunitarios,planta,habitaciones,banos,metros,fecha_construccion,situacion,cusec,precio_m2,...,trastero,piscina_comunitaria,cuarto_de_basura,zonas_comunes,piscina,garaje,tejado,calefaccion,jardin,aire_acondicionado
0,160.0,0.0,9.0,2.0,1.0,45.0,1960.0,3.0,2.807915e+09,167909.090909,...,0,0,0,0,0,0,0,0,0,0
1,125.0,0.0,1.0,2.0,1.0,67.0,1966.0,1.0,2.816101e+09,NaN,...,1,0,0,0,0,0,0,0,0,0
2,NaN,0.0,2.0,2.0,1.0,62.0,1960.0,3.0,2.807915e+09,136690.000000,...,0,0,0,0,0,0,0,0,0,0
3,159.0,0.0,9.0,3.0,1.0,78.0,1983.0,3.0,2.805806e+09,NaN,...,0,0,0,0,0,0,0,0,0,0
4,170.0,0.0,6.0,3.0,1.0,84.0,1980.0,3.0,2.805802e+09,NaN,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,239.0,71.0,2.0,3.0,2.0,78.0,1958.0,2.0,8.019031e+08,236071.428571,...,0,0,0,0,0,0,0,0,0,0
200,239.0,71.0,2.0,2.0,2.0,78.0,1958.0,2.0,8.019031e+08,236071.428571,...,0,0,0,0,0,0,0,0,0,0
201,300.0,30.0,0.0,4.0,2.0,100.0,NaN,3.0,8.019010e+08,477259.797980,...,1,0,0,0,0,0,0,0,0,0
202,237.0,183.0,3.0,2.0,1.0,53.0,1930.0,3.0,8.019021e+08,346777.777778,...,0,0,0,0,0,0,0,0,0,0


In [6]:


def codifica_texto_mas_frecuente(x):
    if x.empty or x.isnull().all():
        return ''
    else: 
        x = x.dropna()
        return x.mode()
    

    




def limpia_car(df):
    df.estado=df.estado.apply(codifica_estado)
    df['NS']=df.orientacion
    df['EO']=df.orientacion

    df['NS']=df['orientacion'].apply(codifica_NS)
    df['EO']=df['orientacion'].apply(codifica_EO)
    df.drop(['orientacion'], axis=1, inplace= True)
    
    
    df = df.groupby('uv', as_index=False).agg(
    {
    'estado': 'max',
    'habitaciones': 'max',
    'banos': 'max',
    'metros': 'max',
    'fecha_construccion': 'max',
    'NS': codifica_texto_mas_frecuente,
    'EO': codifica_texto_mas_frecuente,
    'situacion': 'max',
    'portal': 'max',
    'ascensor': 'max',
    'parque_infantil': 'max',
    'terraza': 'max',
    'trastero': 'max',
    'piscina_comunitaria': 'max',
    'cuarto_de_basura': 'max',
    'zonas_comunes': 'max',
    'piscina': 'max',
    'garaje': 'max',
    'tejado': 'max',
    'calefaccion': 'max',
    'jardin': 'max',
    'aire_acondicionado': 'max',
    }
    )     
    df.drop(df.loc[df['estado']==3].index, inplace=True)
    #eliminamos los estado==3 (normalmente deberían ser los vendidos)
    df.set_index('uv', inplace=True)
    return df



def limpia_web(df):   
    df = df.groupby('uv', as_index=False).agg(
    {
    'tipo_vivienda': codifica_texto_mas_frecuente,
    'precio': 'max',
    'gastos_comunitarios': 'max',
    }
    )     
    #limpiamos el df fijando tipo de vivienda, poniendo el precio de la vivienda en miles de euros K, pasando estado a 1 sin reformar, 2 reformado, 3 vendido. usamos int
    df.tipo_vivienda = df.tipo_vivienda.apply(codifica_tipo_vivienda)
    df.precio =  df.precio.fillna(0)
    df.precio =  (df.precio/1000)

    df.gastos_comunitarios =  df.gastos_comunitarios.fillna(0).astype(int)
    #renombramos las nuevas variables
    df.columns=['uv', 'tipo_vivienda', 'precio_k', 'gastos_comunitarios']
    #las coordenadas vienen multiplicadas por 100
    #2do incluit este ajuste en el json
    df.set_index('uv', inplace=True)
    return df

def limpia_geo(df):
    df = df.groupby('uv', as_index=False).agg(
        {
        'latitud': 'max',
        'longitud': 'max',
        'pais': 'max',
        'ciudad': codifica_texto_mas_frecuente,
        'distrito': codifica_texto_mas_frecuente,
        'calle': codifica_texto_mas_frecuente,
        'barrio': codifica_texto_mas_frecuente,
        'planta': 'max',
        }
    )     

    df.set_index('uv', inplace=True)

    df[['latitud', 'longitud']]=df[['latitud', 'longitud']]/100
    return df

def une_df(df_car, df_web, df_geo):
    df=df_car.join(df_geo, how='outer').join(df_web, how='outer')
    #al hacer el join pueden aparecer nan que rellenamos con ceros y nos aseguramos de que mantenga el formato int.
    df.fillna(int(0), inplace= True)
    df.precio_k = df.precio_k.astype(int)
    return df
